# HashMap

TODO: deal with load factor and re-bucketing.

In [23]:
import java.util.List;

public class HashMap<TKey, TValue> {
    private static class Entry<TKey, TValue> {
        private TKey key;
        private TValue value;
        
        public Entry(TKey key, TValue value) {
            this.key = key;
            this.value = value;
        }
        
        public TKey getKey() {
            return key;
        }
        
        public TValue getValue() {
            return value;
        }
        
        void setValue(TValue value) {
            this.value = value;
        }
    }
    
    private List<List<Entry<TKey, TValue>>> buckets;
    
    private static final int HASH_SIZE = 8;
    
    public HashMap() {
        buckets = new ArrayList<>(Collections.nCopies(2 << HASH_SIZE, null));
    }
    
    public void put(TKey key, TValue value) {
        int index = indexFromHash(key.hashCode());
        if (buckets.get(index) == null) {
            buckets.set(index, new ArrayList<>());
        }
        
        Optional<Entry<TKey, TValue>> entry = buckets.get(index).stream().filter(e -> e.key.equals(key)).findFirst();
        if (entry.isEmpty()) {
            buckets.get(index).add(new Entry<>(key, value));
        }
        else {
            entry.get().setValue(value);
        }
    }
    
    public TValue get(TKey key) {
        int index = indexFromHash(key.hashCode());
        if (buckets.get(index) == null) {
            return null;
        }
        Optional<Entry<TKey, TValue>> entry = buckets.get(index).stream().filter(e -> e.key.equals(key)).findFirst();
        if (entry.isEmpty()) {
            return null;
        }
        
        return entry.get().getValue();
    }
    
    private int indexFromHash(int hash) {
        return ((1 << HASH_SIZE) - 1) & hash;
    }
    
    public int size() {
        return buckets.stream().mapToInt(b -> b == null ? 0 : b.size()).sum();
    }
}

public class HashMapTest {
    public static void main() {
        HashMap<String, String> m = new HashMap<>();
        m.put("Bob", "Cat");
        m.put("Bob", "Dog");
        m.put("Charlie", "Horse");
        
        System.out.println(m.get("Bob"));
        System.out.println(m.get("Charlie"));
        System.out.println(m.size());
    }
}
HashMapTest.main();

Dog
Horse
2


# PerimiterIterator

I wanted to see if this was an easier to way to do the types of problems where you swirl around a matrix in a spiral.  I'm not sure it helped that much given how much code it is.

In [15]:
// to use within a matrix whirlpool rotation type problem, just anchor (0, 0) to the upper left of the layer in the matrix
public class PerimiterIterator {
    private static enum Side {
        TOP,
        LEFT,
        BOTTOM,
        RIGHT
    };
    
    int width; // inclusive
    int height; // inclusive
    
    int index; // 0-based, top-left, clockwise
    int offset;
    
    boolean done;
    boolean clockwise = true;
    
    public PerimiterIterator(int height, int width, boolean clockwise) {
        this(height, width, 0, clockwise);
    }
    
    public PerimiterIterator(int height, int width, int offset, boolean clockwise) {
        if (width < 1 || height < 1) {
            throw new IllegalArgumentException("Height and width must be positive!");
        }
        
        this.width = width;
        this.height = height;
        
        this.offset = offset;
        if (offset < 0) {
            this.offset = Math.abs(this.offset) % perimiter();
            this.offset = perimiter() - this.offset;
        }
        else {
            this.offset = this.offset % perimiter();
        }
        
        this.index = this.offset;
        
        this.clockwise = clockwise;
    }
    
    public PerimiterIterator(int height, int width) {
        this(height, width, 0, true);
    }
    
    public PerimiterIterator(int height, int width, int offset) {
        this(height, width, offset, true);
    }
    
    private int perimiter() {
        if (width == 1 && height == 1) {
            return 1;
        }
        
        return 2 * width + 2 * height - 4;
    }
    
    private int topLeft() {
        return 0;
    }
    
    private int topRight() {
        return width - 1;
    }
    
    private int bottomRight() {
        return width + height - 2;
    }
    
    private int bottomLeft() {
        return 2 * width + height - 3;
    }
    
    private Side getSide() {
        if (index < topRight()) {
            return Side.TOP;
        }
        if (index < bottomRight()) {
            return Side.RIGHT;
        }
        if (index < bottomLeft()) {
            return Side.BOTTOM;
        }
        
        return Side.LEFT;
    }
    
    public void next() {
        if (clockwise) {
            index++;
        }
        else {
            index--;
            if (index < 0) {
                index += perimiter();
            }
        }
        
        index = index % perimiter();
        if (index == offset) {
            done = true;
        }
    }
    
    public int row() {
        switch (getSide()) {
            case TOP:
                return 0;
            case RIGHT:
                return index - topRight();
            case BOTTOM:
                return height - 1;
            default:
                return height - 1 - (index - bottomLeft());
        }
    }
    
    public int col() {
        switch (getSide()) {
            case TOP:
                return index - topLeft();
            case RIGHT:
                return width - 1;
            case BOTTOM:
                return width - 1 - (index - bottomRight());
            default:
                return 0;
        }
    }
    
    public boolean done() {
        return done;
    }
    
    public static void main() {
        PerimiterIterator iter = new PerimiterIterator(4, 3, false);
        while (!iter.done()) {
            System.out.printf("(%d, %d)\n", iter.row(), iter.col());
            iter.next();
        }
    }
}

PerimiterIterator.main();

(0, 0)
(1, 0)
(2, 0)
(3, 0)
(3, 1)
(3, 2)
(2, 2)
(1, 2)
(0, 2)
(0, 1)


# QuickSort

In [34]:
import java.util.Arrays;

class QuickSort {
    private static void swap(int[] arr, int index1, int index2) {
        int temp = arr[index1];
        arr[index1] = arr[index2];
        arr[index2] = temp;
    }
    
    private static void quicksort(int[] arr, int start, int stop) {
        if (Math.abs(stop - start) < 2) {
            return;
        }
        
        int pivotIndex = stop - 1;
        int nextGreater = pivotIndex - 1;
        int nextLess = 0;
        
        while (nextLess <= nextGreater) {
            if (arr[nextLess] < arr[pivotIndex]) {
                ++nextLess;
            }
            else {
                swap(arr, nextLess, nextGreater);
                --nextGreater;
            }
        }
        swap(arr, nextLess, pivotIndex);
        
        quicksort(arr, start, nextLess);
        quicksort(arr, nextLess + 1, stop);
    }
    
    public static void quicksort(int[] arr) {
        quicksort(arr, 0, arr.length);
    }
    
    public static void main(String[] args) {
        int[] arr = {5, 3, 1, 2, 4};
        quicksort(arr);
        
        System.out.println(Arrays.toString(arr));
    }
}

QuickSort.main(null);

[1, 2, 3, 4, 5]


# LinkedList

In [13]:
import java.util.Iterator;

public class LinkedList<T> implements Iterable<T> {
    private static class Node<T> {
        T value;
        
        Node<T> next;
        Node<T> previous;
        
        public Node(T value) {
            this.value = value;
        }
        
        public Node(T value, Node<T> next, Node<T> previous) {
            this.value = value;
            this.next = next;
            this.previous = previous;
        }
        
        @Override
        public String toString() {
            return value.toString();
        }
    }
    
    private Node<T> head;
    private Node<T> tail;
    
    public int size() {
        Node<T> cur = head;
        int count = 0;
        while (cur != null) {
            ++count;
            cur = cur.next;
        }
        
        return count;
    }
    
    private Node<T> seek(int index) {
        // could store size and choose to go backward as optimization
        
        Node<T> cur = head;
        while (index > 0) {
            if (cur == null) {
                throw new IndexOutOfBoundsException();
            }
            cur = cur.next;
            --index;
        }
        if (cur == null) {
            throw new IndexOutOfBoundsException();
        }
        
        return cur;
    }
    
    public T get(int index) {
        return seek(index).value;
    }
    
    public void add(T value) {
        if (head == null) {
            head = new Node<T>(value);
            tail = head;
        }
        else {
            tail.next = new Node<T>(value, null, tail);
            tail = tail.next;
        }
    }
    
    // TODO: properly account for head or tail being different (oops)
    public void add(int index, T value) {
        Node<T> node = seek(index);
        
        Node<T> newNode = new Node<T>(value, node, node.previous);
        node.previous = newNode;
        newNode.previous.next = newNode;
    }
    
    public void set(int index, T value) {
        seek(index).value = value;
    }
    
    public int indexOf(T value) {
        Node<T> cur = head;
        int index = 0;
        while (cur != null) {
            if (cur.value.equals(value)) {
                return index;
            }
            ++index;
            cur = cur.next;
        }
        
        return -1;
    }
    
    public int lastIndexOf(T value) {
        Node<T> cur = tail;
        int index = size() - 1;
        while (cur != null) {
            if (cur.value.equals(value)) {
                return index;
            }
            --index;
            cur = cur.previous;
        }
        
        return -1;
    }
    
    @Override
    public String toString() {
        List<String> elems = new ArrayList<>();
        for (T val : this) {
            elems.add(val.toString());
        }
        return String.format("[%s]", String.join(",", elems));
    }
    
    @Override
    public Iterator<T> iterator() {
        Node<T> head = this.head; // needs to be effectively final
        
        return new Iterator<T>() {
            Node<T> current = head;
            
            public boolean hasNext() {
                return current != null;
            }
            
            public T next() {
                if (current == null) {
                    throw new IndexOutOfBoundsException();
                }
                
                T value = current.value;
                current = current.next;
                return value;
            }
        };
    }
    
    public static void main() {
        LinkedList<Integer> l = new LinkedList<>();
        l.add(10);
        l.add(20);
        l.add(30);
        l.add(1, 15);
        l.set(3, 25);
        
        System.out.println(l.size());
        System.out.println(l.get(0));
        System.out.println(l.get(1));
        System.out.println(l.indexOf(15));
        System.out.println(l.lastIndexOf(15));
        System.out.println();
        
        System.out.println(l);
        l.forEach(System.out::println);
    }
}

LinkedList.main();

4
10
15
1
1

[10,15,20,25]
10
15
20
25
